In [1]:
!pip install PyTDC

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 39.8 MB/s eta 0:00:00
  Created wheel for PyTDC: filename=PyTDC-0.4.1-py3-none-any.whl size=140644 sha256=37af6acc5e2275ff51b273a704f9169ba138c4059a22689d9f43379da596ebf7
  Stored in directory: /root/.cache/pip/wheels/14/b7/b8/9d9e1442129743359b0507234ccc0beb67b47173faccd57d54
Successfully built PyTDC


In [2]:
import tqdm
import numpy as np
import pandas as pd
import plotly.express as px
from tdc.multi_pred import DTI
import plotly.figure_factory as ff

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
data = DTI(name = 'Davis')
split = data.get_split()

Downloading...
  8%|▊         | 1.60M/21.4M [00:00<00:03, 5.42MiB/s]

## Data Provision

In [ ]:
new_data=data.get_data()
# new_data['weights']=1/new_data['Y']
# new_data=new_data.sample(frac=0.5,weights='weights')
y_max=new_data['Y'].max()
y_min=new_data['Y'].min()
new_data['Y']=(new_data['Y']-y_min)/(y_max-y_min)
# new_data['Y']=-np.log10(new_data['Y']/1e9)
new_data=new_data.sample(frac=1)
new_data=new_data.reset_index(drop=True)

## Some basic Analysis

In [ ]:
new_data['Drug_l']=new_data.Drug.apply(len)
new_data['Target_l']=new_data.Target.apply(len)
new_data[['Drug_l','Target_l','Y']].describe()

,Drug_l,Target_l,Y
count,25772.000000,25772.000000,25772.000000
mean,54.176471,744.849604,0.755811
std,10.962637,372.813592,0.399002
min,32.000000,244.000000,0.000000
25%,45.000000,479.000000,0.377499
50%,53.000000,632.000000,1.000000
75%,61.250000,954.000000,1.000000
max,81.000000,2549.000000,1.000000


In [ ]:
new_data[['Drug',"Target"]].tail()

,Drug,Target
25767,Cc1cc2c(F)c(Oc3ncnn4cc(OCC(C)O)c(C)c34)ccc2[nH]1,MTAVYMNGGGLVNPHYARWDRRDSVESGCQTESSKEGEEGQPRQLT...
25768,Cc1[nH]c(C=C2C(=O)Nc3ccc(F)cc32)c(C)c1C(=O)NCC...,MHTGGETSACKPSSVRLAPSFSFHAAGLQMAGQMPHSHQYSDRRQP...
25769,O=C(NOCC1CC1)c1ccc(F)c(F)c1Nc1ccc(I)cc1Cl,MELQAARACFALLWGCALAAAAAAQGKEVVLLDFAAAGGELGWLTH...
25770,Nc1nc(N)c2nc(-c3cccc(O)c3)c(-c3cccc(O)c3)nc2n1,MFQASMRSPNMEPFKQQKVEDFYDIGEELGSGQFAIVKKCREKSTG...
25771,Cc1nc(Nc2ncc(C(=O)Nc3c(C)cccc3Cl)s2)cc(N2CCN(C...,MVDMGALDNLIANTAYLQARKPSDCDSKELQRRRRSLALPGLQGCA...


In [ ]:
fig = px.histogram(new_data['Y'], nbins=200,marginal="box")

# Show the plot
fig.show()

## Let's create the tokenizer

In [ ]:
def tokenize(input_string):
  return [ord(char) for char in input_string]
def encode(input_string,max_length=128,padding=True):
  tokens=tokenize(input_string)
  if len(tokens)>max_length:
    tokens=tokens[:max_length]
  if (len(tokens)<max_length) & padding:
    tokens.extend([0 for _ in range(max_length-len(tokens))])
  return tokens
def decode(input_tokens):
  return ''.join(list(map(lambda x:chr(x), input_tokens)))

In [ ]:
l_tokenizer=encode('z',padding=False)[0]+1

## Now, let's create the dataset object

In [ ]:
class DTIA_Dataset(Dataset):
    def __init__(self, df,drug_max_length,target_max_length):
        self.df = df
        self.dml=drug_max_length
        self.tml=target_max_length
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row=self.df.iloc[idx]
        input_drug=torch.tensor(encode(row['Drug'],max_length=self.dml))
        input_target=torch.tensor(encode(row['Target'],max_length=self.tml))
        y=torch.tensor(row['Y'],dtype=torch.float32)
        return {'input_drug':input_drug,
         'input_target': input_target,
         'y':y}

In [ ]:
dml=45
tml=700

In [ ]:
l=int(new_data.shape[0]*0.8)
train_p=DTIA_Dataset(new_data[:l],drug_max_length=45,target_max_length=700)
test_p=DTIA_Dataset(new_data[l:],drug_max_length=45,target_max_length=700)

In [ ]:
train_loader=DataLoader(train_p,batch_size=32,shuffle=True)
test_loader=DataLoader(test_p,batch_size=32)

## Now a model

In [ ]:
def create_mask(size,distance):
    indices = torch.arange(size).view(1, -1)
    matrix = torch.abs(indices - indices.t()) <= distance
    return matrix.int()
class SingleHeadAttention(torch.nn.Module):
    def __init__(self, qkv_dim,embed_dim, dropout_rate=0.1):
        super(SingleHeadAttention, self).__init__()
        self.qkv_dim=qkv_dim
        self.embed_dim = embed_dim
        self.W_q = torch.nn.Linear(qkv_dim, embed_dim, bias=False)
        self.W_k = torch.nn.Linear(qkv_dim, embed_dim, bias=False)
        self.W_v = torch.nn.Linear(qkv_dim, embed_dim, bias=False)
        self.dp = nn.Dropout(dropout_rate)
    def forward(self, query, key, value, mask=None):
        Q = self.W_q(query) #Q,KVe
        Q=self.dp(Q)
        K = self.W_k(key)   #KV,KVe
        K=self.dp(K)
        V = self.W_v(value) #KV,Ve
        V=self.dp(V)
        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.qkv_dim, dtype=torch.float32)) #Q,KV
        if mask is not None:
            attention_scores = attention_scores.masked_fill(mask == 0, float('-inf')) #Q,KV
        attention_weights = F.softmax(attention_scores, dim=-1) #Q,KV
        attended_values = torch.matmul(attention_weights, V) #Q,Ve
        return attended_values, attention_weights


class MultiHeadAttention(torch.nn.Module):
    def __init__(self, embed_size, num_heads,output_size,dropout_rate=0.1):
        super(MultiHeadAttention, self).__init__()
        self.embed_size = embed_size
        self.num_heads = num_heads
        self.attention_heads = torch.nn.ModuleList([SingleHeadAttention(embed_size,embed_size,dropout_rate) for _ in range(num_heads)])
        self.fc_out = torch.nn.Linear(num_heads * embed_size, output_size)

    def forward(self, query, key, value, mask=None):
        head_outputs = [attention(query, key, value, mask)[0] for attention in self.attention_heads]
        concatenated_output = torch.cat(head_outputs, dim=-1)
        output = self.fc_out(concatenated_output)
        return output

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
drug_mask=create_mask(45,5).to(device)
target_mask=create_mask(700,10).to(device)
# target_mask1=create_mask(700,10).to(device)
class Drug_Model_att(torch.nn.Module):
    def __init__(self, embed_dim=16,dim1=32):
        super(Drug_Model_att, self).__init__()
        self.embeddings = nn.Embedding(l_tokenizer, embed_dim)
        self.pos_embedding = nn.Embedding(45, embed_dim)
        self.att1=MultiHeadAttention(embed_dim,2,embed_dim)
        self.att2=MultiHeadAttention(embed_dim,2,embed_dim)
        self.att3=MultiHeadAttention(embed_dim,2,embed_dim)

    def forward(self, input_ids):
        x = self.embeddings(input_ids)
        x = x + self.pos_embedding(torch.arange(input_ids.size(1)).to(device))
        x=self.att1(x,x,x,drug_mask)
        x=self.att2(x,x,x,drug_mask)
        x=self.att3(x,x,x,drug_mask)
        x=torch.transpose(x,1,2)
        x = torch.squeeze(F.adaptive_avg_pool1d(x, 1),2)
        return x


class Attention_Block(torch.nn.Module):
  def __init__(self,embed_dim,n_att,dropout_rate=0.2):
        super(Attention_Block,self).__init__()
        self.atts=[MultiHeadAttention(embed_dim,2,embed_dim,dropout_rate).to(device) for _ in range(n_att)]
  def forward(self,x,mask=None):
    for layer in self.atts:
          x=layer(x,x,x,mask)
    return x

class Target_Model_att(torch.nn.Module):
    def __init__(self, embed_dim=16,dim1=32):
        super(Target_Model_att, self).__init__()
        self.embeddings = nn.Embedding(l_tokenizer, embed_dim)
        self.pos_embedding = nn.Embedding(700, embed_dim)
        self.att_b1=Attention_Block(embed_dim,2)
        self.pool1=nn.MaxPool1d(10)
        self.att_b2=Attention_Block(embed_dim,2)

        # self.fpool=nn.AvgPool1d(700)

    def forward(self, input_ids):
        x = self.embeddings(input_ids)
        x = x + self.pos_embedding(torch.arange(input_ids.size(1)).to(device))
        x=self.att_b1(x,target_mask)
        x=torch.transpose(x,1,2)
        x=self.pool1(x)
        x=torch.transpose(x,1,2)
        x=self.att_b2(x)
        x=torch.transpose(x,1,2)
        x = torch.squeeze(F.adaptive_max_pool1d(x, 1),2)
        return x


class DTIA_Model(torch.nn.Module):
    def __init__(self, embed_dim=32,dim1=64):
        super(DTIA_Model, self).__init__()
        self.drug_encoder=Drug_Model_att(embed_dim=embed_dim,dim1=dim1)
        self.target_encoder=Target_Model_att(embed_dim=embed_dim,dim1=dim1)
        self.dense1 = nn.Linear(embed_dim*2, 1024)
        self.dp1=nn.Dropout(0.2)
        self.dense2 = nn.Linear(1024, 1024)
        self.dp2=nn.Dropout(0.2)
        self.dense3 = nn.Linear(1024, 512)
        self.dp3=nn.Dropout(0.2)
        self.fdense = nn.Linear(512, 1)


    def forward(self, drug_inputs,target_inputs):
        drug_encoded=self.drug_encoder(drug_inputs)
        target_encoded=self.target_encoder(target_inputs)
        x=torch.concat((drug_encoded,target_encoded),dim=-1)
        x = F.relu(self.dp1(self.dense1(x)))
        x = F.relu(self.dp2(self.dense2(x)))
        x = F.relu(self.dp3(self.dense3(x)))
        x=self.fdense(x)

        return x

In [ ]:

model=DTIA_Model(embed_dim=64,dim1=32)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device='cpu'
model.to(device)

DTIA_Model(
  (drug_encoder): Drug_Model_att(
    (embeddings): Embedding(123, 64)
    (pos_embedding): Embedding(45, 64)
    (att1): MultiHeadAttention(
      (attention_heads): ModuleList(
        (0-1): 2 x SingleHeadAttention(
          (W_q): Linear(in_features=64, out_features=64, bias=False)
          (W_k): Linear(in_features=64, out_features=64, bias=False)
          (W_v): Linear(in_features=64, out_features=64, bias=False)
          (dp): Dropout(p=0.1, inplace=False)
        )
      )
      (fc_out): Linear(in_features=128, out_features=64, bias=True)
    )
    (att2): MultiHeadAttention(
      (attention_heads): ModuleList(
        (0-1): 2 x SingleHeadAttention(
          (W_q): Linear(in_features=64, out_features=64, bias=False)
          (W_k): Linear(in_features=64, out_features=64, bias=False)
          (W_v): Linear(in_features=64, out_features=64, bias=False)
          (dp): Dropout(p=0.1, inplace=False)
        )
      )
      (fc_out): Linear(in_features=128, out_

## Now, training.

In [ ]:
def train(num_epochs=200):
    criterion= nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
    best_test_loss = float('inf')

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_total=0.0
        test_loss=0.0
        test_total=0.0

        for batch in tqdm.tqdm(train_loader):
            input_drug=batch['input_drug'].to(device)
            input_target=batch['input_target'].to(device)
            y=batch['y'].to(device)
            optimizer.zero_grad()
            outputs = model(input_drug,input_target).view(-1,)
            loss = criterion(outputs,y)
            train_loss += loss.item()
            train_total+=outputs.size(0)
            loss.backward()
            optimizer.step()
        avg_train_loss = train_loss / len(train_loader)



        model.eval()
        with torch.no_grad():
            for batch in test_loader:
                input_drug=batch['input_drug'].to(device)
                input_target=batch['input_target'].to(device)
                y=batch['y'].to(device)
                outputs = model(input_drug,input_target).view(-1,)
                loss = criterion(outputs,y)
                test_total+=outputs.size(0)
                test_loss += loss.item()
        avg_test_loss = test_loss / len(test_loader)

        if avg_test_loss < best_test_loss:
            best_test_loss = avg_test_loss
            torch.save(model.state_dict(), "best_model.pt")
        print(f"Epoch {epoch+1}/{num_epochs}: "
              f"Train Loss: {avg_train_loss:.4f}, Valid Loss: {avg_test_loss:.4f}")

In [ ]:
train()

100%|██████████| 645/645 [00:45<00:00, 14.12it/s]


Epoch 1/200: Train Loss: 0.1343, Valid Loss: 0.1330


100%|██████████| 645/645 [00:45<00:00, 14.17it/s]


Epoch 2/200: Train Loss: 0.1224, Valid Loss: 0.1211


100%|██████████| 645/645 [00:45<00:00, 14.18it/s]


...

100%|██████████| 645/645 [00:44<00:00, 14.58it/s]


Epoch 115/200: Train Loss: 0.0891, Valid Loss: 0.0932


100%|██████████| 645/645 [00:44<00:00, 14.54it/s]


Epoch 116/200: Train Loss: 0.0894, Valid Loss: 0.0962


 48%|████▊     | 312/645 [00:21<00:22, 14.95it/s]

In [ ]:
train()

## VISUAL EVALUATION

In [ ]:
ys=[]
preds=[]
for batch in tqdm.tqdm(test_loader):
    input_drug=batch['input_drug'].to(device)
    input_target=batch['input_target'].to(device)
    y=batch['y'].to(device)
    outputs = model(input_drug,input_target).view(-1,)
    ys.extend(list(y.detach().cpu().numpy()))
    preds.extend(list(outputs.detach().cpu().numpy()))
temp_df=pd.DataFrame({'ys':ys,'preds':preds})
px.scatter(temp_df,x='ys',y='preds')

In [ ]:
temp_df.corr()